## Загрузка библиотек и данных

In [137]:
import pandas as pd
import zipfile
import os
import time
import re




In [2]:
from google.colab import drive
from PIL import Image

In [142]:
!pip install pypdf
from pypdf import PdfReader

In [4]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Создание дф

In [5]:
# чтобы разгрузить код я буду путь к файлам сохранять в эту переменную
PATH = None

In [61]:
# создаем датафрейм
PATH = '/content/drive/MyDrive/Masterskaya_2/English_scores.zip (Unzipped Files)/movies_labels.xlsx'
data = pd.read_excel(PATH)
data.head()

,id,Movie,Level
0,0,10_Cloverfield_lane(2016),B1
1,1,10_things_I_hate_about_you(1999),B1
2,2,A_knights_tale(2001),B2
3,3,A_star_is_born(2018),B2
4,4,Aladdin(1992),A2/A2+


In [62]:
data.shape[0]

241

In [63]:
# читаем файл с субтитрами
PATH = '/content/drive/MyDrive/Masterskaya_2/English_scores.zip (Unzipped Files)/Subtitles_all/Subtitles/10_Cloverfield_lane(2016).srt'
subs = pysrt.open(PATH)


In [64]:
#help (pysrt)

In [65]:
# я посмотрела как будут выглядеть субтитры
subtitle_texts = [subtitle.text for subtitle in subs]
subtitle_texts[:10]

['<font color="#ffff80"><b>Fixed & Synced by bozxphd. Enjoy The Flick</b></font>',
 '(CLANGING)',
 '(DRAWER CLOSES)',
 '(INAUDIBLE)',
 '(CELL PHONE RINGING)',
 "BEN ON PHONE: <i>Michelle,<br/>please don't hang up.</i>",
 "<i>Just talk to me, okay?<br/>I can't believe you just left.</i>",
 '<i>Michelle.</i>',
 '<i>Come back.</i>',
 '<i>Please say something.</i>']

Для того, чтобы получить субтитры в удобноваримом формате, мы пройдемся циклом по всему содержимому папки Subtitles, которая содержит папки с названиями фильмов. Для этого используем модуль os, который позволяет работать с файлами и директориями.

Я выбирала между сохранить в словарь название:субтитры или два списка.
Решила сделать два списка, потому что потом мне нужно будет сравнивать фильмы в дф и в списке фильмов с субтитрами, а это легче сделать из списка, а потом добавить в дф.

In [66]:
PATH = '/content/drive/MyDrive/Masterskaya_2/English_scores.zip (Unzipped Files)/Subtitles_all/'

In [67]:
PATH = '/content/drive/MyDrive/Masterskaya_2/English_scores.zip (Unzipped Files)/Subtitles_all/'

def get_subs():
    subtitles = []
    movie = []

    # Получаем все папки и файлы в указанной папке
    film_list = os.listdir(PATH)

    for folder in film_list:
        for film in os.listdir(os.path.join(PATH, folder)):
            # достаем названия
            last_dot = film.rfind('.')
            movie.append(film[:last_dot])

            # субтитры
            # собираем название путь
            file_to_read = os.path.join(PATH, folder, film)  # Исправленный путь к файлу
            # расшифровываем
            try:
                encoded_film = pysrt.open(file_to_read, encoding='iso-8859-1')
            except:
                encoded_film = pysrt.open(file_to_read, encoding='UTF-8')

            # трансформируем в текст
            subs_text = [film.text for film in encoded_film]
            subtitles.append(subs_text)

    return pd.DataFrame({'subs': subtitles, 'Movie': movie})


Обращаем внимание, что имеются строки в которых точка используется как разделитель, это важно, так как нам нужно сохранить названия фильмов без расширения .srt

---

Делить будем так:
- находим индекс где стоит **последняя** точка
- добавляем строку до этого индекса

В работе цикла возникла ошибка что файл не найден в указанной директории.
Я попробовала открыть отдельно этот файл(вручную прописав путь к нему) и возникла ошибка кодировки. Чтобы решить эти вопросы:
- Поэтому добавила в цикл try-except, чтобы обработать все возможные кодировки
- Создала переменную путь и название, чтобы избежать того, что возникает ошибка из-за некорректной обработки названий.


In [68]:
subs_df = get_subs()
subs_df.head()

,subs,Movie
0,"[- ( walkie-talkie squawks )\n- Rick: Morgan,,...",The Walking Dead-S01E05-Wildfire.English
1,"[- ( people yelling )\n- ( radio chatter ), He...",The Walking Dead-S01E06-TS-19.English
2,"[( birds chirping ), - What?\n- Nothing., It's...",The Walking Dead-S01E04-Vatos.English
3,"[( thunder rumbling ), Merle:\nThat's right. Y...",The Walking Dead-S01E03-Tell It To The Frogs.E...
4,"[- ( birds chirping )\n- ( bugs chittering ), ...",The Walking Dead-S01E02-Guts.English


In [69]:
subs_df.shape[0]

278

Теперь можно обработать названия фильмов, чтобы добавить субтитры к фильмам.
Обрабатывать буду следующими шагами:
- все слова к строчному регистру
- в качестве разделителя только знак нижнего подчеркивания "_"
- удалю все пробелы, знаки между словами, оставлю только цифры.

In [70]:
data.head()

,id,Movie,Level
0,0,10_Cloverfield_lane(2016),B1
1,1,10_things_I_hate_about_you(1999),B1
2,2,A_knights_tale(2001),B2
3,3,A_star_is_born(2018),B2
4,4,Aladdin(1992),A2/A2+


In [71]:
def transform_values(df):
    # заменяем пробелы на underscore
    df = df.applymap(lambda x: x.replace(' ', '_') if isinstance(x, str) else x)

    # заменяем дефис на underscore
    df = df.applymap(lambda x: x.replace('-', '_') if isinstance(x, str) else x)

    # добавляем underscore между словом и цислом
    df = df.applymap(lambda x: '_'.join(part for part in x.split() if not part.isdigit()) if isinstance(x, str) else x)
    # приводим к строчному регистру
    df = df.applymap(lambda x: x.lower() if isinstance(x, str) else x)

    return df


In [72]:
# почистим названия фильмов в дата
data = transform_values(data)
data.head()

,id,Movie,Level
0,0,10_cloverfield_lane(2016),b1
1,1,10_things_i_hate_about_you(1999),b1
2,2,a_knights_tale(2001),b2
3,3,a_star_is_born(2018),b2
4,4,aladdin(1992),a2/a2+


In [73]:
data.columns

Index(['id', 'Movie', 'Level'], dtype='object')

In [74]:
subs_df = transform_values(subs_df)
subs_df.head()

,subs,Movie
0,"[- ( walkie-talkie squawks )\n- Rick: Morgan,,...",the_walking_dead_s01e05_wildfire.english
1,"[- ( people yelling )\n- ( radio chatter ), He...",the_walking_dead_s01e06_ts_19.english
2,"[( birds chirping ), - What?\n- Nothing., It's...",the_walking_dead_s01e04_vatos.english
3,"[( thunder rumbling ), Merle:\nThat's right. Y...",the_walking_dead_s01e03_tell_it_to_the_frogs.e...
4,"[- ( birds chirping )\n- ( bugs chittering ), ...",the_walking_dead_s01e02_guts.english


In [75]:
subs_df['Movie'] = transform_values(subs_df[['Movie']])
subs_df['Movie'].head()

0             the_walking_dead_s01e05_wildfire.english
1                the_walking_dead_s01e06_ts_19.english
2                the_walking_dead_s01e04_vatos.english
3    the_walking_dead_s01e03_tell_it_to_the_frogs.e...
4                 the_walking_dead_s01e02_guts.english
Name: Movie, dtype: object

In [76]:
subs_df.head()

,subs,Movie
0,"[- ( walkie-talkie squawks )\n- Rick: Morgan,,...",the_walking_dead_s01e05_wildfire.english
1,"[- ( people yelling )\n- ( radio chatter ), He...",the_walking_dead_s01e06_ts_19.english
2,"[( birds chirping ), - What?\n- Nothing., It's...",the_walking_dead_s01e04_vatos.english
3,"[( thunder rumbling ), Merle:\nThat's right. Y...",the_walking_dead_s01e03_tell_it_to_the_frogs.e...
4,"[- ( birds chirping )\n- ( bugs chittering ), ...",the_walking_dead_s01e02_guts.english


Сделаю тип объединения outer так как могут удалиться дубликаты, а я хочу сохранить все наблюдения и отдельно обработать дубликаты чтобы избежать ошибок.

In [77]:
data = pd.merge (subs_df,data,
                 on = 'Movie',
                 how = 'outer',
                 indicator = True)
data.head()

,subs,Movie,id,Level,_merge
0,"[- ( walkie-talkie squawks )\n- Rick: Morgan,,...",the_walking_dead_s01e05_wildfire.english,116.0,a2,both
1,"[- ( people yelling )\n- ( radio chatter ), He...",the_walking_dead_s01e06_ts_19.english,117.0,a2,both
2,"[( birds chirping ), - What?\n- Nothing., It's...",the_walking_dead_s01e04_vatos.english,115.0,a2,both
3,"[( thunder rumbling ), Merle:\nThat's right. Y...",the_walking_dead_s01e03_tell_it_to_the_frogs.e...,114.0,a2,both
4,"[- ( birds chirping )\n- ( bugs chittering ), ...",the_walking_dead_s01e02_guts.english,113.0,a2,both


In [78]:
# приведем названия столбцов к каноническому виду
data.columns = ['subs', 'movie', 'id', 'level', '_merge']
data.head()

,subs,movie,id,level,_merge
0,"[- ( walkie-talkie squawks )\n- Rick: Morgan,,...",the_walking_dead_s01e05_wildfire.english,116.0,a2,both
1,"[- ( people yelling )\n- ( radio chatter ), He...",the_walking_dead_s01e06_ts_19.english,117.0,a2,both
2,"[( birds chirping ), - What?\n- Nothing., It's...",the_walking_dead_s01e04_vatos.english,115.0,a2,both
3,"[( thunder rumbling ), Merle:\nThat's right. Y...",the_walking_dead_s01e03_tell_it_to_the_frogs.e...,114.0,a2,both
4,"[- ( birds chirping )\n- ( bugs chittering ), ...",the_walking_dead_s01e02_guts.english,113.0,a2,both


In [79]:
data['_merge'].value_counts()

both          233
left_only      49
right_only      8
Name: _merge, dtype: int64

видим что у нас остались 49 фильмов только из датафрейма subs_df и 8 только из data.
очевидно, что для 49 фильмов у нас нет разметки по уровню и для 8 нет субтитров.

In [80]:
data.loc[(data['_merge']=='left_only') | (data['_merge']=='right_only')]

,subs,movie,id,level,_merge
38,"[On the clothes we wear,, the people who make ...",the.true.cost.2015.bluray.720p.700mb.ganool.com,NaN,NaN,left_only
39,"[Yes., Ah, there you are. Come in., I'm practi...","crown,_the_s01e05___smoke_and_mirrors.en",NaN,NaN,left_only
40,"[[woman speaks French] Your Majesty., [planes ...","crown,_the_s01e07___scientia_potentia_est.en.sdh",NaN,NaN,left_only
41,[[music plays]\n<i>...and then I swing you aro...,"crown,_the_s01e10___gloriana.en.sdh",NaN,NaN,left_only
42,"[[child]<i> Gina! Gina! Ella!</i>, -Gina! Cari...","crown,_the_s01e03___windsor.en.sdh",NaN,NaN,left_only
43,"[- Fuel on.\n- Fuel on., Chocks are in positio...","crown,_the_s01e04___act_of_god.en",NaN,NaN,left_only
44,"[[fanfare plays], [man] <i>More than half a mi...","crown,_the_s01e06___gelignite.en.sdh",NaN,NaN,left_only
45,"[[coughing], [coughing continues], [spits], [e...","crown,_the_s01e01___wolferton_splash.en.sdh",NaN,NaN,left_only
46,"[<i>...and then I swing you around there.</i>,...","crown,_the_s01e10___gloriana.en",NaN,NaN,left_only
47,"[<i>âª If your sweetheart âª</i>, <i>âª Sen...","crown,_the_s01e09___assassins.en.sdh",NaN,NaN,left_only


In [155]:

reader = PdfReader('/content/drive/MyDrive/Masterskaya_2/Oxford_CEFR_level/American_Oxford_3000_by_CEFR_level.pdf')
number_of_pages = len(reader.pages)
page = reader.pages[0]
text = page.extract_text

In [157]:
os.listdir('/content/drive/MyDrive/Masterskaya_2/Oxford_CEFR_level')

['American_Oxford_5000_by_CEFR_level.pdf',
 'The_Oxford_5000_by_CEFR_level.pdf',
 'The_Oxford_3000_by_CEFR_level.pdf',
 'American_Oxford_3000_by_CEFR_level.pdf']

нужно либо маску разделитель уровней прописать либо предварительно вручную разграничить по уровням, потому что пока непонятно как работать с этим форматом text

## Предобработка и анализ

In [81]:
data.dtypes

subs        object
movie       object
id         float64
level       object
_merge    category
dtype: object

## Пропуски

На данном этапе заполню пропуски заглушкой 'no_subs', все пропуски возникли по понятной мне причине и их немного. Оставлю я их для того чтобы потом заполнить на стадии обогащения датасеты. А сейчас мне нужно просто получить baseline модель и такие данные сгодятся

In [82]:
plug = 'no_info'

In [83]:
data = data.fillna(plug)
data.isna().sum()

subs      0
movie     0
id        0
level     0
_merge    0
dtype: int64

## Дупликаты

In [121]:
data['movie'].duplicated().sum()

4

In [124]:
duplicates = data[data['movie'].duplicated()]
duplicates

,subs,movie,level,unique_words
151,[Advertise your product or brand here<br/>cont...,inside_out(2015),b1,"[talk, Hey, Everything, pro, planned, iEasy, f..."
175,"[EMT 1:\nAll right, people,\nlet's move! Let's...",powder(1995),b1,"[talk, Hey, Everything, Bullshit, definitely, ..."
185,"[[CROWD CHEERING IN DISTANCE], LElGH ANNE: The...",the_blind_side(2009),b1,"[butt, Hey, talk, tote, Legendary, Everything,..."
228,"[<font color=""#D900D9"">(AIRPLANE APPROACHES)</...",the_terminal(2004),"a2/a2+,_b1","[talk, procedure, Hey, Everything, section, br..."


Мы увидели только ошибочные дупликаты, удалять их не будем.

In [130]:
data['unique_words'].duplicated().sum()

26

In [131]:
data[data['unique_words'].duplicated()]

,subs,movie,level,unique_words
151,[Advertise your product or brand here<br/>cont...,inside_out(2015),b1,"[talk, Hey, Everything, pro, planned, iEasy, f..."
175,"[EMT 1:\nAll right, people,\nlet's move! Let's...",powder(1995),b1,"[talk, Hey, Everything, Bullshit, definitely, ..."
185,"[[CROWD CHEERING IN DISTANCE], LElGH ANNE: The...",the_blind_side(2009),b1,"[butt, Hey, talk, tote, Legendary, Everything,..."
228,"[<font color=""#D900D9"">(AIRPLANE APPROACHES)</...",the_terminal(2004),"a2/a2+,_b1","[talk, procedure, Hey, Everything, section, br..."
260,[],suits_s04e12_engsub,c1,[]
261,[],suits_s04e02_engsub,c1,[]
262,[],suits_s04e06_engsub,c1,[]
263,[],suits_s04e03_engsub,c1,[]
264,[],suits_s04e11_engsub,c1,[]
265,[],suits_s04e07_engsub,c1,[]


### Предварительная обработка субтитров

In [84]:
data.head()

,subs,movie,id,level,_merge
0,"[- ( walkie-talkie squawks )\n- Rick: Morgan,,...",the_walking_dead_s01e05_wildfire.english,116.0,a2,both
1,"[- ( people yelling )\n- ( radio chatter ), He...",the_walking_dead_s01e06_ts_19.english,117.0,a2,both
2,"[( birds chirping ), - What?\n- Nothing., It's...",the_walking_dead_s01e04_vatos.english,115.0,a2,both
3,"[( thunder rumbling ), Merle:\nThat's right. Y...",the_walking_dead_s01e03_tell_it_to_the_frogs.e...,114.0,a2,both
4,"[- ( birds chirping )\n- ( bugs chittering ), ...",the_walking_dead_s01e02_guts.english,113.0,a2,both


- перевести значения в признаке subs в формат str
- нужно создать дополнительный столбец, чтобы превратить субтитры в кортеж из слов
- удалить символы кроме пробела и '', так как по длине фраз так же можно сделать предположение о сложности текста
- привести слова к первоначальной форме (ед число, настоящее время)
- привести все слова к нижнему регистру



In [85]:
# пробуем заменить символы
data['subs'] = data['subs'].replace('\n', '\t', regex=True)
data['subs'][0][:5]

['- ( walkie-talkie squawks )\n- Rick: Morgan,',
 "I don't know if you're out there.",
 "I don't know\nif you can hear me.",
 "Maybe you're listening right now.",
 'I hope so.']

Видим, что остались символы \n без пробела, но на данном этапе не получилось их убрать не используя регулярные выражения.


---

Создадим новый признак, который будет содержать кортеж из уникальных слов, содержащихся в субтитрах к фильму.

data['sub'] содержит субтитры, которые представлены в формате список списков.
Каждый список (фильм) содержит список предложений, каждое предложение имеет формат str.
сначала нам нужно все эти строки (подсписки) соединить в одну строку, где каждый символ(разделен) пробелом.

Поэтому сначала мы получим слова из каждого подсписка в формате списка

  `[sub_words = []

    for sublist in data_column:

        for word in sublist.split():
        
            sub_words.append(word)]`.

Теперь нам нужно список собрать в строку, для этого объединим все элементы этого списка

```
' '.join([word for lists in word_list)
```

Теперь мы имеем субтитры записанные в строковом формате, которые можно почистить от знаков препинаний и создать кортеж вместо списков.

Текст должен быть обработан от символов, иначе получается слишком много слов за счет того что Hello! и Hello считаются разными словами.



In [169]:
def extract_words(data_column):
    sub_words = []
    for sublist in data_column:
        for word in sublist.split():
            sub_words.append(word.lower())

    sub_words = ' '.join(sub_words)
    pattern = r'[^\w\s]'
    sub_words_cleaned = re.sub(pattern, '', sub_words)
    unique_words = sub_words_cleaned.split()
    return unique_words

In [170]:
data['unique_words'] = data['subs'].apply(extract_words)
data['unique_words'][5][:10]

['bugs',
 'chittering',
 'brakes',
 'squeak',
 'engine',
 'stops',
 'trunk',
 'clicks',
 'bird',
 'cawing']

У нас получился новый признак data['unique_words'] который содержит уникальные слова, использованные в каждом фильме.

In [117]:
data.head()

,subs,movie,level,unique_words
0,"[- ( walkie-talkie squawks )\n- Rick: Morgan,,...",the_walking_dead_s01e05_wildfire.english,a2,"[talk, Hey, Leave, Go, loudly, gun, Item, scra..."
1,"[- ( people yelling )\n- ( radio chatter ), He...",the_walking_dead_s01e06_ts_19.english,a2,"[Shit, talk, Hey, volunteered, scan, Go, gun, ..."
2,"[( birds chirping ), - What?\n- Nothing., It's...",the_walking_dead_s01e04_vatos.english,a2,"[rocks, talk, Hey, gravity, paper, kid, Surviv..."
3,"[( thunder rumbling ), Merle:\nThat's right. Y...",the_walking_dead_s01e03_tell_it_to_the_frogs.e...,a2,"[talk, Hey, Paltry, dropped, Go, gun, Everythi..."
4,"[- ( birds chirping )\n- ( bugs chittering ), ...",the_walking_dead_s01e02_guts.english,a2,"[Shit, leaves, Hey, dropped, bell, Surviving, ..."


Вывод:
Мы создали датасет, содержащий следующие колонки:
 - subs, id, movie, level, _merge, unique_words.

Все данные относятся к типу object, числовые признаки отсутствуют.

В ходе соединения

Предобработка включала в себя:
- замена пропущенных значений заглушкой
Сделали предварительную очистку субтитров:
-


# Подготовка модели

In [115]:
#Удалим лишние признаки
data = data.drop(['id', '_merge'], axis = 1)
data.head()

,subs,movie,level,unique_words
0,"[- ( walkie-talkie squawks )\n- Rick: Morgan,,...",the_walking_dead_s01e05_wildfire.english,a2,"[talk, Hey, Leave, Go, loudly, gun, Item, scra..."
1,"[- ( people yelling )\n- ( radio chatter ), He...",the_walking_dead_s01e06_ts_19.english,a2,"[Shit, talk, Hey, volunteered, scan, Go, gun, ..."
2,"[( birds chirping ), - What?\n- Nothing., It's...",the_walking_dead_s01e04_vatos.english,a2,"[rocks, talk, Hey, gravity, paper, kid, Surviv..."
3,"[( thunder rumbling ), Merle:\nThat's right. Y...",the_walking_dead_s01e03_tell_it_to_the_frogs.e...,a2,"[talk, Hey, Paltry, dropped, Go, gun, Everythi..."
4,"[- ( birds chirping )\n- ( bugs chittering ), ...",the_walking_dead_s01e02_guts.english,a2,"[Shit, leaves, Hey, dropped, bell, Surviving, ..."
